In [153]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [154]:
hClusterData = pd.read_csv('./Data/hierarchicalClustering.csv')
kClusterData = pd.read_csv('./Data/kmeansClustering.csv')

clusterDataframes = [hClusterData,kClusterData] 

clusterData = pd.DataFrame()
clusterData['PLAYER'] = hClusterData['PLAYER']
for x in clusterDataframes:
    clusterData = clusterData.merge(x,on="PLAYER",how="outer",suffixes=("","_delme"))
clusterData = clusterData[[c for c in clusterData.columns if not c.endswith('_delme')]]
clusterData = clusterData[['PLAYER','TEAM','hCLUSTER','kCLUSTER','PC1','PC2','PC3','PC4','PC5']]

From here, we are going to interpret the clusters based off the player's hierachial cluster. 

The K-Means Cluster is used as a means to double confirm the cluster that the player has been placed in from Hierachial clustering. 

In [155]:
clusterData.head()

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
0,Stephen Curry,GSW,7,2,-1.796374,1.688838,-1.400482,-1.139984,-0.863846
1,Bradley Beal,WAS,7,2,-0.961425,1.890887,-0.959048,-2.053901,-1.001628
2,Damian Lillard,POR,7,2,-2.044473,2.020990,-0.322098,-2.096059,0.532567
3,Joel Embiid,PHI,1,1,1.454258,3.964915,-1.741855,0.934012,3.536845
4,Giannis Antetokounmpo,MIL,1,2,0.057459,2.931893,0.811177,-0.717473,0.305921


In [156]:
clusterData[clusterData['hCLUSTER']==1].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
3,Joel Embiid,PHI,1,1,1.454258,3.964915,-1.741855,0.934012,3.536845
4,Giannis Antetokounmpo,MIL,1,2,0.057459,2.931893,0.811177,-0.717473,0.305921
7,Zion Williamson,NOP,1,2,0.632637,2.009273,-0.098219,-1.240897,-0.701223
8,Kevin Durant,BKN,1,1,-0.663843,2.234169,-1.257294,-0.374588,1.630744
12,Nikola Jokic,DEN,1,1,0.650120,3.573820,-0.073705,2.973424,2.085747
16,LeBron James,LAL,1,1,-0.866422,2.366130,0.206285,0.394382,1.356265
17,Kawhi Leonard,LAC,1,1,-0.964686,2.277506,-0.292408,0.454101,1.447641
18,Karl-Anthony Towns,MIN,1,1,1.004751,2.783768,-1.441664,1.137774,1.245552
22,Julius Randle,NYK,1,1,-0.459298,2.638682,-0.523151,1.428700,0.502668
25,Nikola Vucevic,CHI,1,1,0.551852,2.277766,-1.772889,2.813408,2.341676


From hCLUSTER 1, we can see that the players generally have high PC2 and a low PC3 value.  
This means that these players are offensive minded stars that are not focusing on assist and can shoot the ball well

In [157]:
clusterData[clusterData['hCLUSTER']==2].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
88,Derrick Rose,NYK,2,6,-1.233110,0.532550,0.700740,-0.286923,1.295217
127,Jalen Brunson,DAL,2,6,-0.997725,0.111372,0.392462,-0.245198,1.128338
147,Immanuel Quickley,NYK,2,6,-1.108979,-0.385465,0.156526,-0.583252,0.987599
149,Lou Williams,ATL,2,6,-1.141651,0.120539,0.595077,-0.954513,1.610352
165,Ty Jerome,OKC,2,6,-0.979790,-0.313963,0.146182,-0.019084,0.785541
174,Monte Morris,DEN,2,6,-1.112014,-0.333945,0.372414,0.376555,0.618136
217,George Hill,PHI,2,6,-0.622539,-0.484019,0.488041,0.269295,1.104954
224,Cameron Payne,PHX,2,6,-1.132937,-0.358270,0.723676,0.209466,1.639853
226,Gabriel Deck,OKC,2,6,0.256301,-0.003771,1.583459,-0.594520,1.586606
236,Tyrese Maxey,PHI,2,6,-0.692363,-0.677807,0.585188,-1.263084,0.982752


From hCLUSTER 2, we can see that the players generally have a low PC1, PC2, PC3 value while they have a high PC4 and PC5 value.  
This implies that 


In [158]:
clusterData[clusterData['hCLUSTER']==3].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
29,Russell Westbrook,WAS,3,2,-1.478353,2.678823,2.280958,0.579597,0.029424
32,Jimmy Butler,MIA,3,2,-0.753911,2.360084,1.658997,0.447088,-0.419183
34,Jamal Murray,DEN,3,2,-1.344194,1.056358,-0.249939,-0.414594,-0.351091
35,Malcolm Brogdon,IND,3,2,-1.461679,1.111222,-0.106822,-0.444910,-0.488897
43,Victor Oladipo,MIA,3,2,-1.155006,0.821678,0.191992,-0.419942,-1.043985
45,Fred VanVleet,TOR,3,2,-1.837493,0.947927,0.529860,1.034133,-1.748902
49,Kemba Walker,OKC,3,2,-1.666446,0.606886,0.277720,-0.303991,0.207736
58,Jrue Holiday,MIL,3,2,-1.050190,1.080975,0.867629,0.216948,-0.520004
61,Darius Garland,CLE,3,2,-1.712335,0.561464,0.678463,-0.213270,-1.236063
62,Kyle Lowry,TOR,3,2,-1.461524,1.105544,0.506330,1.145256,-0.947144


In [159]:
clusterData[clusterData['hCLUSTER']==4].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
19,Jaylen Brown,BOS,4,2,-0.573945,1.271380,-1.195050,-0.370226,-0.768442
28,Jerami Grant,DET,4,2,-0.315535,1.216136,-1.314390,-0.994177,-0.222363
36,Christian Wood,HOU,4,1,1.133153,1.427141,-1.225099,-0.125974,-0.241839
38,Terry Rozier,CHA,4,2,-1.214664,0.523969,-0.869066,0.110038,-1.214003
44,Gordon Hayward,CHA,4,2,-0.598293,1.070024,-0.860755,-0.119770,-1.133700
46,Malik Beasley,MIN,4,4,-0.505571,0.090446,-2.044909,-0.666529,-1.168482
51,Michael Porter Jr.,DEN,4,4,0.568858,0.512300,-2.262140,0.042992,-0.302759
54,Norman Powell,POR,4,2,-0.701969,0.336447,-1.479756,-0.873136,-1.311956
55,Andrew Wiggins,GSW,4,2,-0.171100,0.915944,-0.984095,-0.209375,-1.034321
59,John Collins,ATL,4,1,1.244740,1.192308,-1.478417,0.496078,-0.184725


In [160]:
clusterData[clusterData['hCLUSTER']==5].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
42,Kristaps Porzingis,DAL,5,1,1.141808,1.483195,-1.901751,0.767645,1.588728
97,Al Horford,BOS,5,1,0.355814,0.525123,-0.404360,1.768285,1.624930
106,LaMarcus Aldridge,BKN,5,1,0.752771,0.735419,-1.804807,0.876358,2.573829
111,Marcus Morris Sr.,LAC,5,1,0.115147,0.198728,-1.997367,0.349463,1.506494
112,Carmelo Anthony,POR,5,1,-0.049185,0.457660,-1.605717,-0.873307,2.875865
114,Danilo Gallinari,ATL,5,1,0.299469,0.568750,-2.027336,-0.574524,1.873519
125,Alec Burks,NYK,5,4,-0.678806,0.140863,-0.521593,-0.082192,1.179218
135,Kevin Love,CLE,5,1,0.368558,0.450281,-0.969503,0.956294,1.944535
148,Rudy Gay,SAS,5,1,0.105741,0.023617,-0.934139,0.132464,1.423198
157,Serge Ibaka,LAC,5,1,1.175090,0.520182,-0.583308,0.958053,1.559177


In [161]:
clusterData[clusterData['hCLUSTER']==6].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
99,Joe Harris,BKN,6,5,-0.368235,-0.336491,-2.152880,1.064586,-1.339056
109,Mikal Bridges,PHX,6,5,-0.137733,-0.176777,-1.253444,0.910426,-1.558916
115,Duncan Robinson,MIA,6,5,-0.755120,-0.631427,-2.011470,1.777811,-1.460546
121,Kyle Kuzma,LAL,6,5,0.086510,-0.089904,-0.779083,0.669235,-0.568049
129,Seth Curry,PHI,6,5,-1.136296,-0.281587,-1.186997,0.763204,0.817067
143,Davis Bertans,WAS,6,5,-0.271745,-0.989615,-2.039334,0.855283,-0.734828
155,Armoni Brooks,HOU,6,4,-0.432224,-0.825705,-1.151703,0.023995,-0.132770
156,Lonnie Walker IV,SAS,6,4,-0.706820,-0.707704,-0.660388,-0.437814,0.223143
162,Reggie Bullock,NYK,6,5,-0.621633,-0.784944,-1.781586,1.569833,-0.250922
164,Patty Mills,SAS,6,6,-1.309724,-0.880399,-0.765507,0.390708,0.824863


In [162]:
clusterData[clusterData['hCLUSTER']==7].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
0,Stephen Curry,GSW,7,2,-1.796374,1.688838,-1.400482,-1.139984,-0.863846
1,Bradley Beal,WAS,7,2,-0.961425,1.890887,-0.959048,-2.053901,-1.001628
2,Damian Lillard,POR,7,2,-2.044473,2.020990,-0.322098,-2.096059,0.532567
5,Luka Doncic,DAL,7,2,-1.549115,2.905345,1.006958,-1.943889,0.667388
6,Zach LaVine,CHI,7,2,-1.187504,1.602980,-0.905715,-1.654194,-1.388656
9,Kyrie Irving,BKN,7,2,-1.288144,1.777996,-0.271992,-0.795589,-0.039272
10,Jayson Tatum,BOS,7,2,-0.993738,1.877769,-0.746730,-1.046118,0.242550
11,Donovan Mitchell,UTA,7,2,-1.748258,1.558954,-0.056625,-1.131059,-0.022730
13,Devin Booker,PHX,7,2,-1.158058,1.717389,-0.539096,-1.089882,-0.167247
14,Trae Young,ATL,7,2,-2.511069,1.809443,1.653033,-1.838264,-0.414789


In [163]:
clusterData[clusterData['hCLUSTER']==8].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
82,Clint Capela,ATL,8,3,2.507234,1.424498,1.276307,0.324999,-1.068164
86,Andre Drummond,LAL,8,3,2.088552,1.930031,1.746615,0.392241,0.300261
94,Rudy Gobert,UTA,8,3,2.518455,1.708477,0.833973,0.395935,-1.291610
108,Montrezl Harrell,LAL,8,3,1.720492,1.159199,0.362372,-0.675509,0.829293
122,Jarrett Allen,CLE,8,3,2.094858,1.270383,0.258647,0.157527,-0.138774
152,Wendell Carter Jr.,ORL,8,3,1.611594,1.010061,0.071330,0.247712,-0.102908
154,Enes Freedom,POR,8,3,2.026510,1.065574,0.483230,-0.476075,0.628268
171,Mason Plumlee,DET,8,3,1.491115,1.278029,2.208110,1.209609,-0.535330
204,Robin Lopez,WAS,8,3,1.698691,0.260538,-0.034124,-0.569919,0.787440
205,Ivica Zubac,LAC,8,3,1.960377,0.770025,0.403150,0.012410,0.046451


In [164]:
clusterData[clusterData['hCLUSTER']==9].head(10)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
77,Terrence Ross,ORL,9,4,-0.685977,0.006784,-0.987195,-1.129041,0.585273
92,Jaren Jackson Jr.,MEM,9,4,1.219692,0.755330,-0.923659,-0.927620,-0.896619
98,Marvin Bagley III,SAC,9,4,1.244819,0.359260,-0.858261,-1.135904,0.178224
103,Chris Boucher,TOR,9,4,1.276573,0.529508,-1.319640,-0.264160,-0.285298
120,Cole Anthony,ORL,9,2,-0.661769,0.470104,0.834126,-1.831349,-0.053433
132,Brook Lopez,MIL,9,4,1.263942,0.498533,-1.788633,-0.349016,0.328783
139,Jordan Poole,GSW,9,4,-0.699681,-0.523048,-0.955734,-0.789471,0.336159
141,Malik Monk,CHA,9,4,-0.582311,-0.455206,-0.848320,-1.128949,0.409061
144,James Wiseman,GSW,9,3,1.473751,0.134283,-0.520338,-0.877579,-0.705318
146,Bobby Portis,MIL,9,4,0.911789,0.245062,-0.916307,0.263543,0.474326


In [165]:
clusterData[clusterData['TEAM']=='PHI'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
3,Joel Embiid,PHI,1,1,1.454258,3.964915,-1.741855,0.934012,3.536845
47,Tobias Harris,PHI,1,1,-0.314828,1.457216,-0.551250,0.436435,0.518464
217,George Hill,PHI,2,6,-0.622539,-0.484019,0.488041,0.269295,1.104954
236,Tyrese Maxey,PHI,2,6,-0.692363,-0.677807,0.585188,-1.263084,0.982752
95,Ben Simmons,PHI,3,2,-0.364463,1.372504,1.738063,1.283216,-0.816352
387,Matisse Thybulle,PHI,3,5,-0.018439,-1.384660,0.519737,1.374085,-1.531619
118,Shake Milton,PHI,4,2,-0.914452,0.118838,0.236582,-0.588436,-0.212558
129,Seth Curry,PHI,6,5,-1.136296,-0.281587,-1.186997,0.763204,0.817067
192,Danny Green,PHI,6,5,-0.223731,-0.541560,-1.028377,1.706622,-1.412198
202,Furkan Korkmaz,PHI,6,4,-0.535197,-0.900606,-0.957029,-0.073674,-0.194878


In [166]:
clusterData[clusterData['TEAM']=='BKN'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
8,Kevin Durant,BKN,1,1,-0.663843,2.234169,-1.257294,-0.374588,1.630744
247,Mike James,BKN,2,6,-1.412953,-0.231566,1.782153,-0.508811,2.311095
211,Bruce Brown,BKN,4,5,0.636269,-0.348984,0.597111,0.266105,-1.291971
294,Nic Claxton,BKN,4,3,1.584650,-0.220050,0.834245,-0.083420,-0.795961
106,LaMarcus Aldridge,BKN,5,1,0.752771,0.735419,-1.804807,0.876358,2.573829
160,Blake Griffin,BKN,5,1,0.148917,0.577317,-0.254686,0.760166,1.735801
99,Joe Harris,BKN,6,5,-0.368235,-0.336491,-2.152880,1.064586,-1.339056
197,Landry Shamet,BKN,6,5,-0.564286,-0.939922,-1.258419,0.123564,-0.534972
304,Timothe Luwawu-Cabarrot,BKN,6,5,-0.031809,-1.225115,-0.564561,0.098902,-0.735012
332,Tyler Johnson,BKN,6,6,-0.711940,-1.456001,0.013984,1.173047,1.225679


In [167]:
clusterData[clusterData['TEAM']=='MIL'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
4,Giannis Antetokounmpo,MIL,1,2,0.057459,2.931893,0.811177,-0.717473,0.305921
39,Khris Middleton,MIL,1,2,-0.983294,1.236211,-0.478597,-0.190529,0.360914
287,Jeff Teague,MIL,2,6,-0.960286,-0.303514,1.095171,-0.815774,2.411367
58,Jrue Holiday,MIL,3,2,-1.050190,1.080975,0.867629,0.216948,-0.520004
389,P.J. Tucker,MIL,3,5,0.184123,-1.013061,0.212538,1.566682,-0.929437
169,Donte DiVincenzo,MIL,6,5,-0.493085,-0.301030,0.070379,1.108281,-1.024148
179,Bryn Forbes,MIL,6,4,-0.460970,-1.126613,-1.504588,0.024034,1.003023
284,Pat Connaughton,MIL,6,5,0.144785,-0.985106,-0.659089,1.311787,-0.367115
132,Brook Lopez,MIL,9,4,1.263942,0.498533,-1.788633,-0.349016,0.328783
146,Bobby Portis,MIL,9,4,0.911789,0.245062,-0.916307,0.263543,0.474326


In [168]:
clusterData[clusterData['TEAM']=='NYK'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
22,Julius Randle,NYK,1,1,-0.459298,2.638682,-0.523151,1.428700,0.502668
88,Derrick Rose,NYK,2,6,-1.233110,0.532550,0.700740,-0.286923,1.295217
147,Immanuel Quickley,NYK,2,6,-1.108979,-0.385465,0.156526,-0.583252,0.987599
337,Taj Gibson,NYK,3,3,1.306087,-0.128464,1.030106,1.190164,0.421180
60,RJ Barrett,NYK,4,2,-0.462286,0.655184,-0.756969,-0.244709,-1.231360
125,Alec Burks,NYK,5,4,-0.678806,0.140863,-0.521593,-0.082192,1.179218
162,Reggie Bullock,NYK,6,5,-0.621633,-0.784944,-1.781586,1.569833,-0.250922
227,Mitchell Robinson,NYK,8,3,1.875732,0.054960,2.050056,0.795242,-1.938305
350,Nerlens Noel,NYK,8,3,1.745915,0.299653,1.735610,0.302622,-1.813512
175,Elfrid Payton,NYK,9,6,-0.443597,-0.118074,1.306761,-0.433960,0.070589


In [169]:
clusterData[clusterData['TEAM']=='ATL'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
149,Lou Williams,ATL,2,6,-1.141651,0.120539,0.595077,-0.954513,1.610352
360,Brandon Goodwin,ATL,2,6,-0.918400,-1.116080,1.317018,-0.410121,0.630155
59,John Collins,ATL,4,1,1.244740,1.192308,-1.478417,0.496078,-0.184725
71,Bogdan Bogdanovic,ATL,4,5,-1.030647,0.161348,-1.066225,1.050412,-0.766459
85,De'Andre Hunter,ATL,4,4,0.039363,0.688424,-1.206741,-0.532267,-0.091595
140,Kevin Huerter,ATL,4,5,-0.723812,-0.113292,-0.419545,0.711375,-1.111867
151,Cam Reddish,ATL,4,4,-0.052218,-0.313713,-0.635798,-1.401028,-1.490016
114,Danilo Gallinari,ATL,5,1,0.299469,0.568750,-2.027336,-0.574524,1.873519
338,Tony Snell,ATL,6,6,-0.563948,-1.372626,-0.958612,1.476761,1.523883
371,Solomon Hill,ATL,6,5,-0.006788,-1.101848,-0.062398,1.187823,0.232633


In [170]:
clusterData[clusterData['TEAM']=='MIA'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
53,Bam Adebayo,MIA,1,1,0.823731,2.130521,-0.060416,1.497395,0.878431
32,Jimmy Butler,MIA,3,2,-0.753911,2.360084,1.658997,0.447088,-0.419183
43,Victor Oladipo,MIA,3,2,-1.155006,0.821678,0.191992,-0.419942,-1.043985
110,Goran Dragic,MIA,3,6,-0.984440,0.325661,0.293417,0.526555,0.312041
375,Andre Iguodala,MIA,3,6,-0.263949,-0.952529,1.036773,2.617886,0.296294
83,Tyler Herro,MIA,4,2,-0.868575,0.197946,-0.250782,-0.392263,-0.569982
90,Kendrick Nunn,MIA,4,5,-0.792369,-0.105620,-0.781849,0.161026,-0.653990
115,Duncan Robinson,MIA,6,5,-0.755120,-0.631427,-2.011470,1.777811,-1.460546
193,Trevor Ariza,MIA,6,5,0.134459,-0.410040,-0.705955,1.227911,-0.288910
270,Dewayne Dedmon,MIA,8,3,1.844355,0.289141,0.712181,-0.135859,1.532174


In [171]:
clusterData[clusterData['TEAM']=='UTA'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
72,Mike Conley,UTA,3,2,-1.822238,0.809657,0.552309,0.816558,-0.306632
138,Joe Ingles,UTA,3,5,-1.516539,0.114000,0.115928,1.840779,-0.002097
276,Royce O'Neale,UTA,3,5,-0.281239,-0.344239,-0.007818,3.034125,-1.718330
66,Bojan Bogdanovic,UTA,4,4,-0.428320,0.612137,-1.948624,-0.275716,-0.249371
278,Georges Niang,UTA,6,5,0.061950,-1.112492,-1.484589,0.845011,0.570134
11,Donovan Mitchell,UTA,7,2,-1.748258,1.558954,-0.056625,-1.131059,-0.022730
56,Jordan Clarkson,UTA,7,2,-1.007599,0.655328,-0.745872,-1.007955,-0.061320
94,Rudy Gobert,UTA,8,3,2.518455,1.708477,0.833973,0.395935,-1.291610
330,Derrick Favors,UTA,8,3,1.823301,-0.126316,1.413186,-0.323933,-0.562284


In [172]:
clusterData[clusterData['TEAM']=='PHX'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
91,Deandre Ayton,PHX,1,1,2.044541,1.625327,-0.074636,0.943475,0.615642
224,Cameron Payne,PHX,2,6,-1.132937,-0.358270,0.723676,0.209466,1.639853
357,E'Twaun Moore,PHX,2,6,-0.273786,-0.985923,0.922815,-0.322636,1.321976
70,Chris Paul,PHX,3,6,-2.122083,1.523574,1.470628,0.728107,1.890910
213,Dario Saric,PHX,5,1,0.781560,0.123052,-0.550380,0.644467,0.753894
297,Frank Kaminsky,PHX,5,1,0.749029,-0.287992,-0.006457,0.524388,1.480514
109,Mikal Bridges,PHX,6,5,-0.137733,-0.176777,-1.253444,0.910426,-1.558916
178,Jae Crowder,PHX,6,5,-0.089536,-0.287763,-1.041644,2.021551,-0.016822
189,Cameron Johnson,PHX,6,5,-0.237142,-0.893196,-1.075467,1.140030,-0.357192
329,Torrey Craig,PHX,6,4,0.536867,-0.860866,0.014940,0.164479,0.304300


In [173]:
clusterData[clusterData['TEAM']=='DEN'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
12,Nikola Jokic,DEN,1,1,0.650120,3.573820,-0.073705,2.973424,2.085747
174,Monte Morris,DEN,2,6,-1.112014,-0.333945,0.372414,0.376555,0.618136
239,Austin Rivers,DEN,2,6,-1.000468,-0.861110,0.536063,0.549903,0.683975
34,Jamal Murray,DEN,3,2,-1.344194,1.056358,-0.249939,-0.414594,-0.351091
311,Facundo Campazzo,DEN,3,6,-1.211012,-0.887655,0.994281,1.309617,-0.115370
51,Michael Porter Jr.,DEN,4,4,0.568858,0.512300,-2.262140,0.042992,-0.302759
126,Will Barton,DEN,4,5,-0.452258,0.201947,-0.377522,0.051914,-0.349577
131,Aaron Gordon,DEN,4,1,0.250297,0.747509,0.220296,0.330881,0.626436
208,Paul Millsap,DEN,5,1,0.631823,0.143791,-0.159215,0.664703,1.011700
233,JaMychal Green,DEN,5,4,0.802649,-0.417504,-0.641705,0.752949,0.840262


In [174]:
clusterData[clusterData['TEAM']=='LAC'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
17,Kawhi Leonard,LAC,1,1,-0.964686,2.277506,-0.292408,0.454101,1.447641
26,Paul George,LAC,1,2,-1.124663,1.700835,-0.440818,0.277129,-0.033496
321,Yogi Ferrell,LAC,2,6,-1.742404,-1.171864,1.623359,0.787876,2.952196
333,Rajon Rondo,LAC,2,6,-1.150877,-0.433324,2.102664,1.083665,1.957774
111,Marcus Morris Sr.,LAC,5,1,0.115147,0.198728,-1.997367,0.349463,1.506494
157,Serge Ibaka,LAC,5,1,1.175090,0.520182,-0.583308,0.958053,1.559177
209,DeMarcus Cousins,LAC,5,1,0.703386,0.448211,-0.124272,0.936663,1.959627
166,Reggie Jackson,LAC,6,6,-0.752886,-0.221433,-0.089213,0.705049,0.688615
228,Luke Kennard,LAC,6,4,-0.580129,-0.896021,-0.825087,-0.047955,1.178724
234,Nicolas Batum,LAC,6,5,0.026535,-0.493214,-0.460752,2.384621,0.115959


In [175]:
clusterData[clusterData['TEAM']=='DAL'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
127,Jalen Brunson,DAL,2,6,-0.997725,0.111372,0.392462,-0.245198,1.128338
293,Trey Burke,DAL,2,6,-0.807069,-1.104431,0.062599,-0.579221,1.739708
67,Tim Hardaway Jr.,DAL,4,4,-0.323736,0.161204,-2.145172,-1.057120,0.292233
136,Josh Richardson,DAL,4,5,-0.685883,-0.036595,-0.037247,0.114438,-0.614468
340,Willie Cauley-Stein,DAL,4,3,1.456302,-0.382174,0.912142,0.225747,-0.373089
42,Kristaps Porzingis,DAL,5,1,1.141808,1.483195,-1.901751,0.767645,1.588728
181,Dorian Finney-Smith,DAL,6,5,0.061594,-0.396555,-0.756466,1.722247,-0.877353
259,JJ Redick,DAL,6,6,-0.847856,-1.598214,-0.539732,0.147051,1.270494
271,Maxi Kleber,DAL,6,5,0.265003,-0.616352,-1.049920,1.892950,0.280590
392,Nicolo Melli,DAL,6,6,0.127292,-1.276178,0.761561,1.348102,1.406352


In [176]:
clusterData[clusterData['TEAM']=='POR'].sort_values(by=['hCLUSTER']).head(20)

,PLAYER,TEAM,hCLUSTER,kCLUSTER,PC1,PC2,PC3,PC4,PC5
145,Jusuf Nurkic,POR,1,1,1.350834,1.185985,0.315484,0.910330,0.429984
241,Anfernee Simons,POR,2,4,-0.639556,-0.905235,-0.391402,-0.500655,1.726810
54,Norman Powell,POR,4,2,-0.701969,0.336447,-1.479756,-0.873136,-1.311956
282,Derrick Jones Jr.,POR,4,5,0.698969,-0.873444,-0.127665,-0.171590,-1.295249
112,Carmelo Anthony,POR,5,1,-0.049185,0.457660,-1.605717,-0.873307,2.875865
222,Robert Covington,POR,6,5,0.162464,-0.083304,-0.664582,2.150519,-1.349195
2,Damian Lillard,POR,7,2,-2.044473,2.020990,-0.322098,-2.096059,0.532567
27,CJ McCollum,POR,7,2,-1.480921,0.911345,-0.849642,-0.954680,0.768812
154,Enes Freedom,POR,8,3,2.026510,1.065574,0.483230,-0.476075,0.628268
368,Nassir Little,POR,9,4,0.503691,-0.948240,-0.469540,-1.681734,1.758065
